In [1]:
from mltoolbox.basic import *
from mltoolbox.plotting import save_spectrogram

In [2]:
import pyarrow.parquet as pq
from pathlib import Path

In [3]:
PATH = Path('data/')

In [4]:
import os

In [6]:
os.mkdir('data/train_spectograms/')

In [7]:
train = pq.read_pandas(PATH/'train.parquet').to_pandas()

In [8]:
train.iloc[:, :5].head()

,0,1,2,3,4
0,18,1,-19,-16,-5
1,18,0,-19,-17,-6
2,17,-1,-20,-17,-6
3,18,1,-19,-16,-5
4,18,0,-19,-16,-5


In [9]:
from scipy import signal

In [10]:
SAMPLE_LENGTH = 800000
SAMPLE_TIME = 20e-3     # in second
SAMPLING_RATE = SAMPLE_LENGTH / SAMPLE_TIME

In [11]:
def make_spectograms(idx):
    data = train.iloc[:, idx]
    residual = data - data.ewm(alpha=0.01).mean()
    f, t, Sxx = signal.spectrogram(x=residual, fs=SAMPLING_RATE,
                                   window='hamming', nperseg=50000,
                                   nfft=SAMPLE_LENGTH)
    save_spectrogram(Sxx, fpath='data/train_spectograms/' + f'{idx}.jpg',
                     log_col=True, log_freq=True)

In [14]:
import warnings
warnings.simplefilter('ignore')

In [15]:
make_spectograms(0)

Checking Image

![Image](data/train_spectograms/9.jpg)

In [16]:
_ = parallel(make_spectograms, range(0, 10), n_jobs=7)

In [24]:
!ls data/train_spectograms/

0.jpg	 15.jpg  25.jpg  35.jpg  45.jpg  55.jpg  65.jpg  75.jpg  85.jpg  95.jpg
100.jpg  16.jpg  26.jpg  36.jpg  46.jpg  56.jpg  66.jpg  76.jpg  86.jpg  96.jpg
101.jpg  17.jpg  27.jpg  37.jpg  47.jpg  57.jpg  67.jpg  77.jpg  87.jpg  97.jpg
102.jpg  18.jpg  28.jpg  38.jpg  48.jpg  58.jpg  68.jpg  78.jpg  88.jpg  98.jpg
103.jpg  19.jpg  29.jpg  39.jpg  49.jpg  59.jpg  69.jpg  79.jpg  89.jpg  99.jpg
104.jpg  1.jpg	 2.jpg	 3.jpg	 4.jpg	 5.jpg	 6.jpg	 7.jpg	 8.jpg	 9.jpg
10.jpg	 20.jpg  30.jpg  40.jpg  50.jpg  60.jpg  70.jpg  80.jpg  90.jpg
11.jpg	 21.jpg  31.jpg  41.jpg  51.jpg  61.jpg  71.jpg  81.jpg  91.jpg
12.jpg	 22.jpg  32.jpg  42.jpg  52.jpg  62.jpg  72.jpg  82.jpg  92.jpg
13.jpg	 23.jpg  33.jpg  43.jpg  53.jpg  63.jpg  73.jpg  83.jpg  93.jpg
14.jpg	 24.jpg  34.jpg  44.jpg  54.jpg  64.jpg  74.jpg  84.jpg  94.jpg


In [25]:
job_list = [int(i) for i in train.columns.values][104:]

In [26]:
_ = parallel(make_spectograms, job_list, n_jobs=6)